In [1]:
import pandas as pd
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
games = pd.read_csv('nba_data/FiveThirtyEight/nba_elo.csv').filter(['date', 'season', 'playoff', 'team1', 'team2', 'score1', 'score2'])
upcoming_games = games[games.score1.isnull()]
games = games[games.score1.notnull()]

In [3]:
def simulate_game(teamA_mean_pts, teamA_sd_pts, teamA_mean_against, teamA_sd_against, teamB_mean_pts, teamB_sd_pts, teamB_mean_against, teamB_sd_against):
    teamA_score = ( random.gauss(teamA_mean_pts, teamA_sd_pts) + random.gauss(teamB_mean_against, teamB_sd_against) ) / 2
    teamB_score = ( random.gauss(teamB_mean_pts, teamB_sd_pts) + random.gauss(teamA_mean_against, teamA_sd_against) ) / 2
    delta = teamA_score - teamB_score
    return delta

In [4]:
def simulate_matchup(teamA, teamB, n=20000, season=datetime.now().year, only_regular_season=True):
    games_season = games[games.season == season]
    if only_regular_season:
        games_season = games_season[games_season.playoff.isnull()]

    teamA_total_pts = games_season[games_season.team1 == teamA]["score1"].sum() + games_season[games_season.team2 == teamA]["score2"].sum()
    teamB_total_pts = games_season[games_season.team1 == teamB]["score1"].sum() + games_season[games_season.team2 == teamB]["score2"].sum()

    teamA_scores = []
    teamA_against = []

    teamB_scores = []
    teamB_against = []

    for idx, teamA_game in games_season[ (games_season.team1 == teamA) | (games_season.team2 == teamA)].iterrows():
        if teamA_game.team1 == teamA:
            teamA_scores.append(teamA_game.score1)
            teamA_against.append(teamA_game.score2)
        else:
            teamA_scores.append(teamA_game.score2)
            teamA_against.append(teamA_game.score1)

    for idx, teamB_game in games_season[ (games_season.team1 == teamB) | (games_season.team2 == teamB)].iterrows():
        if teamB_game.team1 == teamB:
            teamB_scores.append(teamB_game.score1)
            teamB_against.append(teamB_game.score2)
        else:
            teamB_scores.append(teamB_game.score2)
            teamB_against.append(teamB_game.score1)

    teamA_mean_pts = np.mean(teamA_scores)
    teamA_sd_pts = np.std(teamA_scores)
    teamA_mean_against = np.mean(teamA_against)
    teamA_sd_against = np.std(teamA_against)

    teamB_mean_pts = np.mean(teamB_scores)
    teamB_sd_pts = np.std(teamB_scores)
    teamB_mean_against = np.mean(teamB_against)
    teamB_sd_against = np.std(teamB_against)

    simulations = []
    teamA_wins = 0
    teamB_wins = 0
    ties = 0
    for i in range(n):
        gm = simulate_game(teamA_mean_pts, teamA_sd_pts, teamA_mean_against, teamA_sd_against, teamB_mean_pts, teamB_sd_pts, teamB_mean_against, teamB_sd_against)
        simulations.append(gm)
        if gm > 0:
            teamA_wins += 1
        elif gm < 0:
            teamB_wins += 1
        else:
            ties += 1

    print("{}: {}%\n{}: {}%".format(teamA, (teamA_wins * 100 / n), teamB, (teamB_wins * 100 / n)))
    return simulations
    #return ( (teamA_wins * 100 / n), (teamB_wins * 100 / n) )

In [5]:
simulate_matchup("LAC", "LAL", season=2020)

LAC: 48.785%
LAL: 51.215%


583112,
 -10.213214395520325,
 -10.095174282971087,
 5.84610766767905,
 -15.775596661266619,
 -31.086680779916122,
 2.046054895803465,
 -8.932030310814397,
 -13.923956980108485,
 -4.85596803103995,
 6.201995257027818,
 6.7072605611053575,
 -28.885501144748545,
 19.20648603423983,
 2.3457485742415116,
 16.625294558159567,
 -25.12896036375973,
 5.335546565676566,
 -2.2191325277456997,
 -11.850949821105658,
 -9.423627473433385,
 9.421960063532282,
 -13.425440364434351,
 -0.061026421676587006,
 -2.5671321614157563,
 -0.6682532629583733,
 -4.51249250854363,
 -31.028786896753004,
 -7.171450009980745,
 14.128863325410606,
 -3.354876942170648,
 8.016625425633066,
 13.181218333358075,
 26.61205755996714,
 -10.43562577592354,
 -8.473519821818613,
 -18.56489756948193,
 7.845818390068942,
 -8.547293004697224,
 7.626739339958945,
 -13.86582543279134,
 11.018674544821735,
 20.367905314260312,
 -3.976551599159933,
 -2.316724943403557,
 10.689926190888386,
 -5.354164741954449,
 20.287606019036247,
 11

In [6]:
for idx, matchup in upcoming_games.iterrows():
    print("{} vs {}".format(matchup.team1, matchup.team2))
    simulate_matchup(matchup.team1, matchup.team2)
    print("-" * 30)
print("DONE")

NOP vs UTA
NOP: 42.565%
UTA: 57.435%
------------------------------
LAL vs LAC
LAL: 51.375%
LAC: 48.625%
------------------------------
BRK vs ORL
BRK: 49.87%
ORL: 50.13%
------------------------------
WAS vs PHO
WAS: 45.765%
PHO: 54.235%
------------------------------
POR vs MEM
POR: 49.305%
MEM: 50.695%
------------------------------
MIL vs BOS
MIL: 58.27%
BOS: 41.73%
------------------------------
SAS vs SAC
SAS: 50.685%
SAC: 49.315%
------------------------------
DAL vs HOU
DAL: 54.95%
HOU: 45.05%
------------------------------
DEN vs MIA
DEN: 49.425%
MIA: 50.575%
------------------------------
OKC vs UTA
OKC: 48.235%
UTA: 51.765%
------------------------------
LAC vs NOP
LAC: 61.32%
NOP: 38.68%
------------------------------
IND vs PHI
IND: 48.735%
PHI: 51.265%
------------------------------
TOR vs LAL
TOR: 48.14%
LAL: 51.86%
------------------------------
BRK vs WAS
BRK: 55.36%
WAS: 44.64%
------------------------------
BOS vs POR
BOS: 64.11%
POR: 35.89%
-------------------------

(30,)